# Select 200 random images from the top 20 represented orders

In [1]:
import sys

sys.path.append('..')

In [2]:
import shutil
import sqlite3
from pathlib import Path

import pandas as pd

In [3]:
ROOT_DIR = Path('..')
DATA_DIR = ROOT_DIR / 'data'

DB = DATA_DIR / 'angiosperms.sqlite'

SAMPLE_DIR = DATA_DIR / 'temp' / 'phenology_sample_2022-04-19'

In [4]:
SAMPLE_DIR.mkdir()

In [5]:
sql = """
    select order_, count(distinct coreid) as n
    from targets
    join images using (coreid)
    join angiosperms using (coreid)
    group by order_
    order by n desc
    limit 20
"""
with sqlite3.connect(DB) as cxn:
    df = pd.read_sql(sql, cxn)
df

,order_,n
0,asterales,3147
1,caryophyllales,2318
2,poales,2070
3,lamiales,1446
4,fabales,1235
5,brassicales,1060
6,malpighiales,969
7,boraginales,915
8,fagales,860
9,asparagales,831


In [6]:
sql = """
    select distinct path
    from targets
    join images using (coreid)
    join angiosperms using (coreid)
    where order_ = ?
    order by random()
    limit 200
    """

with sqlite3.connect(DB) as cxn:
    cxn.row_factory = sqlite3.Row
    for _, row in df.iterrows():
        print(row['order_'])
        rows = cxn.execute(sql, (row['order_'],))
        for row in rows:
            row = dict(row)
            src = ROOT_DIR / row['path']
            dst = SAMPLE_DIR / src.name
            shutil.copy(src, dst)

asterales
caryophyllales
poales
lamiales
fabales
brassicales
malpighiales
boraginales
fagales
asparagales
ericales
rosales
myrtales
gentianales
ranunculales
apiales
solanales
alismatales
sapindales
dipsacales
